In [39]:
import os
import pickle
import numpy as np
import pandas as pd
from collections import Counter
import time
import sys
import warnings

#tuning hyperparameters
from bayes_opt import BayesianOptimization
from skopt  import BayesSearchCV 

#graph, plots
import matplotlib.pyplot as plt
import seaborn as sns

#building models
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import roc_auc_score, roc_curve


warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

In [40]:
"""НЕ ЗАБУДЬТЕ:
- отсортировать df по дате.
- перевести значения колонки в int or float.
"""


def occ(x: list):
    c = Counter(x)
    c = dict(sorted(c.items(), key=lambda item: item[1]))
    res = list(c.keys())[-1]

    return res


def days_agg(dataframe: pd.DataFrame, column_name: str, agg_func: str, days: int):
    """Агрегирует данные в выбранной колонке за заданное колчество дней. По сути юзер-френдли 'скользящее окно'.

    params agg_func: модет принимать значение ['mean', 'std', 'sum', 'amplitude', 'occ'].
        - amplitude: max(value) - min(value)
        - occ: максимальное встречающееся значение
            [2,3,4,2,2,2,3]: {2: 4, 3: 2, 4: 1} => вернет 2, максимально встречающееся значение
    params days: ширина окна
    params column_name: что агрегировать
    params dataframe: dataframe

    return: ...
    """

    d = {
        'mean': np.mean,
        'sum': np.sum,
        'std': np.std,
        'amplitude': lambda x: np.max(x) - np.min(x),
        'occ': occ
    }

    agg_f = d[agg_func]
    values = np.array(dataframe[column_name])
    result = []

    for i in range(0, len(values)):
        i_ = i - days
        if i_ < 0:
            i_ = 0
        
        result.append(agg_f(values[i_:i+1]))
    
    return result


In [41]:
def _ts_to_table(idx, time_series, window_size):
    """ Method convert time series to lagged form.
    :param idx: the indices of the time series to convert
    :param time_series: source time series
    :param window_size: size of sliding window, which defines lag
    :return updated_idx: clipped indices of time series
    :return features_columns: lagged time series feature table
    """

    # Convert data to lagged form
    lagged_dataframe = pd.DataFrame({'t_id': time_series})
    vals = lagged_dataframe['t_id']
    for i in range(1, window_size + 1):
        frames = [lagged_dataframe, vals.shift(i)]
        lagged_dataframe = pd.concat(frames, axis=1)

    # Remove incomplete rows
    lagged_dataframe.dropna(inplace=True)

    transformed = np.array(lagged_dataframe)

    # Generate dataset with features
    features_columns = transformed[:, 1:]
    features_columns = np.fliplr(features_columns)

    return idx, features_columns

def convert_wated_codes(value):
    values = list(map(int, map(lambda x: x.strip(), value.split(','))))
    res = 0
    
    for val in values:
        res += water_codes[water_codes['water_code'] == val].reset_index(drop=True).iloc[0][1]
    
    return res

def feature_aggregation(new_df):
    columns = new_df.columns
    columns_drop = []

    if 'discharge' in columns:
        new_df['discharge_mean'] = days_agg(new_df, 'discharge', 'mean', 4)
        columns_drop.append('discharge')
    if 'stage_avg' in columns:
        new_df['stage_avg_amplitude'] = days_agg(new_df, 'stage_avg', 'amplitude', 7)
        new_df['stage_avg_mean'] = days_agg(new_df, 'stage_avg', 'mean', 4)
        columns_drop.append('stage_avg')
    if 'snow_coverage_station' in columns:
        new_df['snow_coverage_station_amplitude'] = days_agg(new_df, 'snow_coverage_station', 'amplitude', 7)
        columns_drop.append('snow_coverage_station')
    if 'snow_height' in columns:
        new_df['snow_height_mean'] = days_agg(new_df, 'snow_height', 'mean', 4)
        new_df['snow_height_amplitude'] = days_agg(new_df, 'snow_height', 'amplitude', 7)
        columns_drop.append('snow_height')
    if 'precipitation' in columns:
        new_df['precipitation_sum'] = days_agg(new_df, 'precipitation', 'sum', 30)
        columns_drop.append('precipitation')
    if 'water_hazard' in columns:
        new_df['water_hazard_sum'] = days_agg(new_df, 'water_hazard', 'sum', 2)
        columns_drop.append('water_hazard')

    new_df.drop(columns_drop, axis=1, inplace=True)

    return new_df

    

In [42]:
ids = [3019, 3027, 3028, 3030, 3035, 3041, 3045, 3230, 3050, 3029]

In [43]:
sample_sub_1 = pd.read_csv('./submissions/sample_submissions/sample_sub_1.csv')
sample_sub_2 = pd.read_csv('./submissions/sample_submissions/sample_sub_2.csv')
sample_sub_3 = pd.read_csv('./submissions/sample_submissions/sample_sub_3.csv')
sample_sub_4 = pd.read_csv('./submissions/sample_submissions/sample_sub_4.csv')

In [45]:
water_codes = pd.read_csv('./data/misc/ref_code_hazard.csv')
water_codes.head()

,water_code,hazard
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [95]:
def predict(id_, sample_sub):
    meteo_prep = pd.read_csv(f'./data/meteo_data/no_gap_meteo_3hour_int_{id_}_wind.csv')
    meteo_prep.drop(['station_id'], inplace=True, axis=1)
    
    meteo = pd.read_csv(f'./data/meteo_data/no_gap_meteo_1day_int_{id_}.csv')
    meteo.drop(['station_id'], inplace=True, axis=1)
    meteo.columns = ['date', 'snow_coverage_station', 'snow_height']
    
    train = pd.read_csv(f'./data/4rd_checkpoint/sub_datasets_no_gaps/no_gaps/no_gap_train_{id_}.csv')
    train = train.set_index('date')
    train['water_hazard'] = train['water_code'].fillna('1').apply(convert_wated_codes)
    train.drop(['stage_min', 'stage_max', 'temp', 'water_code', 'station_id', 'ice_thickness',
                'snow_height', 'place', 'year', 'month', 'day', 'delta_stage_max'], axis=1, inplace=True)
    
    columns = ['0_day', '1_day', '2_day', '3_day', '4_day', '5_day', '6_day', '7_day']
    target_column = 'stage_avg'

    a = train[target_column]
    window_size = 8
    idx = np.arange(train.shape[0] - window_size) + 1
    idx, b = _ts_to_table(idx, a, window_size)

    train = train.iloc[:-window_size, :]

    new_df = pd.DataFrame(data=b, columns=columns, index=a.index[:-window_size])
    new_df = pd.concat([train, new_df], axis=1, join='inner')
    new_df.drop(['0_day'], axis=1, inplace=True)
    
    new_df = pd.merge(meteo_prep, new_df, how='inner', on=['date'])
    new_df = pd.merge(meteo, new_df, how='inner', on=['date'])
    new_df.drop(['wind_direction', 'wind_speed_aver'], axis=1, inplace=True)
    
    predictions = []
    columns = ['1_day', '2_day', '3_day', '4_day', '5_day', '6_day', '7_day']
    
    for index in range(0, len(sample_sub['date']), 7):
        mini_df = new_df[new_df['date'] < sample_sub['date'][index]]
        mini_df = feature_aggregation(mini_df)
        mini_df.drop(['date'], inplace=True, axis=1)

        features = np.array(mini_df.drop(columns, axis=1))
        features_test = np.array(mini_df.drop(columns, axis=1).iloc[-1, :]).reshape(1, -1)
        target = np.array(mini_df[columns])

        model = MultiOutputRegressor(lgb.LGBMRegressor(random_state=42), n_jobs=-1)
        model.fit(features, target)
        pred = model.predict(features_test)
        predictions.append(pred)
    
    return predictions

In [96]:
station_ids = sample_sub_4['station_id'].unique()
dfs = {}

for id_ in station_ids:
    sample_sub = sample_sub_4[sample_sub_4['station_id'] == id_].reset_index(drop=True)
    predictions = predict(id_, sample_sub)
    sample_sub['delta_stage_max'] = np.concatenate(np.concatenate(predictions))
    dfs[id_] = sample_sub

In [118]:
submission = pd.concat(dfs.values())
stage = []
predictions = submission['delta_stage_max'].reset_index(drop=True)

for i in range(len(predictions)):
    if i % 7 == 0:
        stage.append(predictions[i + 1] - predictions[i])
    else:
        stage.append(predictions[i] - predictions[i - 1])
        
submission['delta_stage_max'] = stage
submission

,year,station_id,month,day,date,delta_stage_max
0,1993,3019,4,111,1993-04-21,0.353349
1,1993,3019,4,112,1993-04-22,0.353349
2,1993,3019,4,113,1993-04-23,-0.793318
3,1993,3019,4,114,1993-04-24,18.742820
4,1993,3019,4,115,1993-04-25,-6.868219
...,...,...,...,...,...,...
247,2013,3230,5,134,2013-05-14,-5.526033
248,2013,3230,5,135,2013-05-15,49.909082
249,2013,3230,5,136,2013-05-16,-24.173369
250,2013,3230,5,137,2013-05-17,-20.934579


In [119]:
submission.to_csv('submission_4_lgbm.csv', index=False)

## MODELING, ENSEMBLING

### FEDOT

In [187]:
from fedot.api.main import Fedot

In [ ]:
predictions = []
columns = ['discharge_mean', 'stage_avg_amplitude', 'stage_avg_mean', 'snow_coverage_station_amplitude',
           'snow_height_mean', 'snow_height_amplitude', 'precipitation_sum', 'water_hazard_sum']


composer_params = {'max_depth': 5,
                   'max_arity': 7,
                   'pop_size': 5,
                   'num_of_generations': 20,
                   'learning_time': 10,
                   'with_tuning': True}

model = Fedot(problem='regression', preset='light', learning_time=13, composer_params=composer_params, seed=42)
model.fit(features=new_df.drop(columns, axis=1), target=columns)
pred = model.predict()
predictions.append(pred.predict)

### LGBM

### LightAutoML

In [180]:
from lightautoml.automl.base import AutoML
from lightautoml.ml_algo.boost_lgbm import BoostLGBM
from lightautoml.ml_algo.tuning.optuna import OptunaTuner
from lightautoml.automl.blend import WeightedBlender
from lightautoml.ml_algo.boost_cb import BoostCB
from lightautoml.ml_algo.linear_sklearn import LinearLBFGS

from lightautoml.pipelines.ml.base import MLPipeline
from lightautoml.reader.base import PandasToPandasReader
from lightautoml.tasks import Task

In [181]:
task = Task('reg')
reader = PandasToPandasReader(task, cv=5, random_state=1)

model1 = BoostLGBM(default_params={'learning_rate': 0.1, 'num_leaves': 128, 'seed': 1, 'num_threads': 5})
params_tuner2 = OptunaTuner(n_trials=100, timeout=100)
model2 = BoostLGBM(default_params={'learning_rate': 0.05, 'num_leaves': 64, 'seed': 2, 'num_threads': 5})
gbm_0 = BoostCB()
gbm_1 = BoostCB()
tuner_0 = OptunaTuner(n_trials=100, timeout=100, fit_on_holdout=True)


pipeline_lvl1 = MLPipeline([model1, (model2, params_tuner2), (gbm_0, tuner_0), gbm_1])
reg_2 = LinearLBFGS()
pipeline_lvl2 = MLPipeline([reg_2])

predictions = []


timer = PipelineTimer(600, mode=2)
automl = AutoML(reader, [
    [pipeline_lvl1],
    [pipeline_lvl2],
], skip_conn=False, blender=WeightedBlender(), timer=timer)
pred = automl.fit_predict(new_df.drop(columns, axis=1), roles={'target': columns})
predictions.append(pred)


In [22]:
sample_sub_4 = pd.read_csv('./submissions/sample_submissions/sample_sub_4.csv')
sample_sub_4

,year,station_id,month,day,date,delta_stage_max
0,1993,3019,4,111,1993-04-21,0
1,1993,3019,4,112,1993-04-22,0
2,1993,3019,4,113,1993-04-23,0
3,1993,3019,4,114,1993-04-24,0
4,1993,3019,4,115,1993-04-25,0
...,...,...,...,...,...,...
2480,2013,3230,5,134,2013-05-14,0
2481,2013,3230,5,135,2013-05-15,0
2482,2013,3230,5,136,2013-05-16,0
2483,2013,3230,5,137,2013-05-17,0


In [29]:
station_ids = sample_sub_4['station_id'].unique()
dfs = {}

for id_ in station_ids:
     dfs[id_] = sample_sub_4[sample_sub_4['station_id'] == id_]

In [2]:


ids = [3019, 3027, 3028, 3030, 3035, 3041, 3045, 3230, 3050, 3029]
results = []


for id_ in ids:
    with open(f'predictions{id_}.pkl', 'rb') as f:
        array = pickle.load(f)
        results.append(array)

In [24]:
np.concatenate(np.concatenate(results[0]))

array([ 36.59279675,  34.31469746,  45.79399691, ..., 438.67104222,
       424.5675937 , 419.17649831])

In [10]:
q = []

for a in results[0]:
    print(type(a[0]))
    q += a[0]

<class 'numpy.ndarray'>


ValueError: operands could not be broadcast together with shapes (0,) (7,) 